In [ ]:
# Drive version:
# !pip uninstall TabularExperimentTrackerClient --y
# !pip install git+https://github.com/DanielWarfield1/TabularExperimentTrackerClient
# !pip uninstall NeuralNetworksTrainingPackage --y
# !pip install git+https://github.com/Bartosz-G/NeuralNetworksTrainingPackage

In [ ]:
%pip uninstall NeuralNetworksTrainingPackage --y
#%pip uninstall TabularExperimentTrackerClient --y
%pip install git+https://github.com/DanielWarfield1/TabularExperimentTrackerClient
%pip install git+https://github.com/Bartosz-G/NeuralNetworksTrainingPackage

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import torch
import time

In [ ]:
import os
from TabularExperimentTrackerClient.ExperimentClient import ExperimentClient

path =  '../creds/'
creds_orch_file = "creds-orch.txt"
creds_openml_file = "creds-openml.txt"



with open(os.path.join(path, creds_orch_file), 'r') as file:
    lines = file.readlines()
    orchname = lines[0].strip()
    orchsecret = lines[1].strip()

with open (os.path.join(path, creds_openml_file), "r") as myfile:
    openMLAPIKey = myfile.read()

ex = ExperimentClient(verbose = True)


ex.define_orch_cred(orchname, orchsecret)
ex.define_opml_cred(openMLAPIKey)

# Colab version
# ex.define_opml_cred_drive('/My Drive/research/non-homogenous-data/creds/creds-openml.txt')
# ex.define_orch_cred_drive('bart', '/My Drive//research/non-homogenous-data/creds/creds-colab.txt')

# 1. Defining the experiment

In [ ]:
experiment_name = 'test_LCN_6'



# LCN and LLN Parameters
depth = {'distribution': 'categorical', 'values':[2,3,4,5,6,7,8,9,10,11,12]}
seed = {'distribution': 'constant', 'value': 42}
drop_type = {'distribution': 'constant', 'value': 'isDropConnect'} # defined by p = 0 or p =/= 0
p = {'distribution': 'categorical', 'values':[0, 0, 0.25, 0.5, 0.75]} # 40% probability of no regularisation
hidden_dim = {'distribution': 'constant', 'value': 1} # Assertion error coming from Net if not 1
anneal = {'distribution': 'constant', 'value': 'none'}
optimizer = {'distribution': 'categorical', 'values': ['SGD', 'SGD', 'SGD', 'SGD', 'AMSGrad']} # 80% of SGD and 20% AMSGrad
batch_size = {'distribution': 'constant', 'value': 64}
epochs = {'distribution': 'constant', 'value': 30}
lr = {'distribution': 'log_uniform', 'min':0.05, 'max':0.2} # yields mean = 0.1082, median 0.1
momentum = {'distribution': 'constant', 'value': 0.9}
no_cuda = {'distribution': 'constant', 'value': False}
lr_step_size = {'distribution': 'categorical', 'value':10}
gamma = {'distribution': 'constant', 'value': 0.1}

# Regression:
back_n_reg = {'distribution': 'categorical', 'values':[0, 1, 2, 3, 4]}

# Classification:
back_n_cls = {'distribution': 'constant', 'value': 0}

In [ ]:
#============================================================
# Locally Constant Networks
#============================================================
LCN_reg_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n_reg,
    'net_type': {'distribution': 'constant', 'value': 'locally_constant'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': optimizer,
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'regression'}
    }

LCN_cls_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n_cls,
    'net_type': {'distribution': 'constant', 'value': 'locally_constant'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': optimizer,
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'classification'}
}

#============================================================
# Locally Linear Networks
#============================================================

LLN_reg_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n_reg,
    'net_type': {'distribution': 'constant', 'value': 'locally_linear'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': optimizer,
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'regression'}
    }


LLN_cls_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n_cls,
    'net_type': {'distribution': 'constant', 'value': 'locally_linear'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': optimizer,
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'classification'}
    }

In [ ]:
model_groups = {
    'LCN_reg':{'model':'LCN_reg', 'hype':LCN_reg_space},
    'LCN_cls':{'model':'LCN_cls', 'hype':LCN_cls_space},
    'LLN_reg':{'model':'LLN_reg', 'hype':LLN_reg_space},
    'LLN_cls':{'model':'LLN_cls', 'hype':LLN_cls_space}
}

ex.def_model_groups(model_groups)

In [ ]:
ex.def_data_groups_opml()
print(f'automatically defined data groups: {ex.data_groups.keys()}')

classification_models = [k for k in model_groups.keys() if '_cls' in k]
regression_models = [k for k in model_groups.keys() if '_reg' in k]


applications = {'opml_reg_purnum_group': regression_models,
                'opml_reg_numcat_group': regression_models,
                'opml_class_purnum_group': classification_models,
                'opml_class_numcat_group': classification_models}

ex.def_applications(applications)
ex.reg_experiment(experiment_name)

In [ ]:
exp_info = ex.experiment_info()
successful_runs = exp_info['successful_runs']
required_runs = exp_info['required_runs']
print('total required runs: {}'.format(required_runs))

# 2. General Data pre-processing

In [ ]:
from NeuralNetworksTrainingPackage.event_handler import dataPreProcessingEventEmitter
from NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing import filterCardinality, quantileTransform, truncateData,balancedTruncateData, oneHotEncodePredictors, oneHotEncodeTargets, toDataFrame, splitTrainValTest, balancedSplitTrainValTest

n_sample = 20000
split = [0.5, 0.25, 0.25]
quantile_transform_distribution='normal'


data_pre_processing = dataPreProcessingEventEmitter()

filter_cardinality = filterCardinality(transform = 'all')
truncate_data = truncateData(n = n_sample, transform = 'all')
balanced_truncate_data = balancedTruncateData(n = n_sample, transform = 'all') # Ensures balance of classes
one_hot_encode_predictors = oneHotEncodePredictors(transform = 'all')
one_hot_encode_targets = oneHotEncodeTargets(transform = 'all')
to_data_frame = toDataFrame(transform = 'all')
split_train_val_test = splitTrainValTest(split = split)
balanced_split_train_val_test = balancedSplitTrainValTest(split = split)
quantile_transform = quantileTransform(output_distribution = quantile_transform_distribution, transform = 'all')


# Transformations will be called in the order they're added to data_pre_processing
data_pre_processing.add_pre_processing_step('regression', filter_cardinality)
data_pre_processing.add_pre_processing_step('regression', truncate_data)
data_pre_processing.add_pre_processing_step('regression', one_hot_encode_predictors)
data_pre_processing.add_pre_processing_step('regression', to_data_frame)
data_pre_processing.add_pre_processing_step('regression', split_train_val_test)
data_pre_processing.add_pre_processing_step('regression', quantile_transform)


data_pre_processing.add_pre_processing_step('classification', filter_cardinality)
data_pre_processing.add_pre_processing_step('classification', balanced_truncate_data)
data_pre_processing.add_pre_processing_step('classification', one_hot_encode_predictors)
data_pre_processing.add_pre_processing_step('classification', one_hot_encode_targets)
data_pre_processing.add_pre_processing_step('classification', to_data_frame)
data_pre_processing.add_pre_processing_step('classification', balanced_split_train_val_test)
data_pre_processing.add_pre_processing_step('classification', quantile_transform)

# 3. Model Specific Data pre-processing

In [ ]:
from NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing import CustomDataset, toPyTorchDatasets

to_pytorch_datasets = toPyTorchDatasets(wrapper = CustomDataset)

# Transformations will be called after general pre-processing steps, and in order they're added
data_pre_processing.add_pre_processing_step('LCN_reg', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LCN_cls', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LLN_reg', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LLN_cls', to_pytorch_datasets)

# 4. Model Metrics

In [ ]:
from metrics.LcnMetrics import LcnMetricsClassification, LcnMetricsRegression

lcn_metrics_regression = LcnMetricsRegression()
lcn_metrics_classification = LcnMetricsClassification()

lcn_metrics = {'regression': lcn_metrics_regression,
               'classification': lcn_metrics_classification}

metric_model_pairs = {
    'LCN_reg': lcn_metrics,
    'LCN_cls': lcn_metrics,
    'LLN_reg': lcn_metrics,
    'LLN_cls': lcn_metrics
}

# 5. Model Training routine

In [ ]:
from models.LcnNetwork import init_LcnNetwork
from training.LcnTrain import LcnTrainingRoutine

lcn_model_and_training = {'model_init':init_LcnNetwork, 'training_routine': LcnTrainingRoutine}

model_training_pairs = {
    'LCN_reg': lcn_model_and_training,
    'LCN_cls': lcn_model_and_training,
    'LLN_reg': lcn_model_and_training,
    'LNN_cls': lcn_model_and_training
}

# 6. Main Experiment loop

In [ ]:
torch_models = ('LCN_reg', 'LCN_cls', 'LLN_reg', 'LLN_cls')

sklearn_models = ('no_sklearn_models_in_this_training')


for i in range(14160):
    print(f'==== Begin run:{i} ====')
    run_info = ex.begin_run_sticky()

    hyperparameters = run_info['hyp']
    model_name = run_info['model']

    task = hyperparameters['task']
    seed = hyperparameters['seed']



    torch.manual_seed(seed)
    np.random.seed(seed)


    print('---- Loading datasets ----')
    X, y, categorical_indicator, attribute_names = ex.opml_load_task(run_info['mtpair_task'])

    # Pre-processing
    data_pre_processing.set_seed_for_all(seed)
    data_pre_processing.set_dataset(X, y, categorical_indicator, attribute_names)
    data_pre_processing.apply(task)
    data_pre_processing.apply(model_name)
    train_data, val_data, test_data = data_pre_processing.get_train_val_test()


    # Getting appropriate metrics
    metrics_calculator = metric_model_pairs[model_name][task]


    match model_name:
        case _ if model_name in sklearn_models:
            pass

        case _ if model_name in torch_models:
            # hyperparameters will be updated with {'input_dim': num_columns_X, 'output_dim':num_columns_Y}
            hyperparameters.update(train_data.get_dims())

            train_batch_size = hyperparameters['batch_size']
            train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=train_batch_size,shuffle= True)
            val_dataloader = torch.utils.data.DataLoader(val_data,batch_size=len(val_data),shuffle= True)
            test_dataloader = torch.utils.data.DataLoader(test_data,batch_size=len(test_data),shuffle= True)


            init_model = model_training_pairs[model_name]['model_init']
            TrainingRoutine = model_training_pairs[model_name]['training_routine']


            model = init_model(**hyperparameters)
            training_routine = TrainingRoutine(**hyperparameters)
            
            training_routine.set_optimizer_scheduler(model)

            start_epoch = 1  # start from epoch 1 or last checkpoint epoch
            total_epochs = hyperparameters['epochs']
            start_time = time.time()

            for epoch in range(start_epoch, total_epochs + start_epoch):
                print(f"----{epoch}th training epoch ----")
                epoch_metrics = {}

                training_routine.scheduler_step(epoch)
                train_loss = training_routine.train(model, train_dataloader)


                if train_loss is None:
                    print('---Stopping training due to loss being nan!---')
                    epoch_metrics = {'train_loss': train_loss, 'epoch': epoch}
                    ex.update_run(epoch_metrics)
                    break

                if epoch == total_epochs:
                    continue

                epoch_metrics.update(train_loss)
                epoch_metrics.update({'epoch': epoch})
                ex.update_run(epoch_metrics)
                print(epoch_metrics)

            else:
                final_metrics = {}
                training_time = time.time()-start_time

                train_metrics = metrics_calculator.get_metrics(model, train_dataloader, hyperparameters, 'train')
                val_all_metrics = metrics_calculator.get_all(model, val_dataloader, hyperparameters, 'val')
                test_all_metrics = metrics_calculator.get_all(model, test_dataloader, hyperparameters, 'test')

                final_metrics.update(train_loss)
                final_metrics.update(train_metrics)
                final_metrics.update(val_all_metrics)
                final_metrics.update(test_all_metrics)
                final_metrics.update({'epoch': epoch})
                final_metrics.update({'epoch_time': training_time})

                ex.update_run(final_metrics)
                print(final_metrics)

    ex.end_run()

## Checking whether the code works as intended

In [ ]:
y

In [ ]:
def get_metrics(model, test_loader, hyperparameters, test_set_name=None):
        if test_set_name:
            assert isinstance(test_set_name, str), "test_set_name must be a string, such as train, val, test"
            
        no_cuda = hyperparameters['no_cuda']    
        use_cuda = not no_cuda and torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")

        with torch.no_grad():
            model.eval()
            test_loss = 0
            dataset_len = 0
            standard_errors_list = []
            target_list = []

            for data, target in test_loader:
                dataset_len += len(target)
                data, target = data.to(device), target.to(device)

                target_list.append(target)

                ###############
                data.requires_grad = True
                if model.net_type == 'locally_constant':
                    output, relu_masks = model(data, p=0, training=False)
                elif model.net_type == 'locally_linear':
                    output, relu_masks = model.normal_forward(data, p=0, training=False)
                ###############

                standard_errors = ((output - target) ** 2)
                # print(standard_errors.shape)
                # print(target.shape)
                standard_errors_list.append(standard_errors)
                test_loss += ((output - target) ** 2).mean().item() * len(target)

            test_loss /= dataset_len
            
            standard_errors_tensor = torch.cat(standard_errors_list, dim=0)
            all_targets = torch.cat(target_list, dim=0)

            
            standard_errors_tensor = torch.squeeze(standard_errors_tensor)
            all_targets = torch.squeeze(all_targets)
            
            print(standard_errors_tensor.shape)
            print(all_targets.shape)


            ss_res = torch.sum(standard_errors_tensor)
            target_mean = torch.mean(all_targets)
            ss_tot = torch.sum((all_targets - target_mean) ** 2)
            r2_score = 1 - (ss_res / ss_tot)
            
            # print(ss_res)
            # print(target_mean)
            # print(ss_tot)
            # print(r2_score)

            RMSE = float(np.sqrt(test_loss))

            quantiles = torch.tensor(self.quantiles, device=device)
            quantile_values = torch.quantile(standard_errors_tensor, quantiles)

            # standard_errors_pd = pd.DataFrame(standard_errors_tensor.cpu().numpy())
            # quantile_dict = standard_errors_pd.quantile(self.quantiles).to_dict()

            quantile_dict = {q: float(v.item()) for q, v in zip(self.quantiles, quantile_values)}

            metrics = {
                'RMSE': RMSE,
                'r2_score': r2_score.cpu().item(),
                'se_quant': quantile_dict}

            if test_set_name:
                assert isinstance(test_set_name, str), "test_set_name must be a string, such as 'train', 'val', 'test'"
                metrics_name = f'{test_set_name}_metrics'
                final_metrics = {metrics_name: metrics}
            else:
                final_metrics = {'metrics': metrics}


            return final_metrics


In [ ]:
metrics = get_metrics(model, train_dataloader, hyperparameters, 'train')

In [ ]:
model = init_model(**hyperparameters)

In [ ]:
X, y, categorical_indicator, attribute_names = ex.opml_load_task('335-361097')

In [ ]:
from copy import deepcopy
#X_c, y_c, categorical_indicator_c, attribute_names_c = deepcopy(X), deepcopy(y), deepcopy(categorical_indicator), deepcopy(attribute_names)
X, y, categorical_indicator, attribute_names = deepcopy(X_c), deepcopy(y_c), deepcopy(categorical_indicator_c), deepcopy(attribute_names_c)

In [ ]:
X_c

In [ ]:
y_c

In [ ]:
categorical_indicator

In [ ]:
data_pre_processing.set_seed_for_all(seed)
data_pre_processing.reset()
data_pre_processing.set_dataset(X, y, categorical_indicator, attribute_names)
data_pre_processing.apply(task)
data_pre_processing.apply(model_name)

In [ ]:
data_pre_processing.reset()

In [ ]:
print(data_pre_processing.val)

In [ ]:
train_data, val_data, test_data = data_pre_processing.get_train_val_test()

In [ ]:
train_data

In [ ]:
X_out, y_out, categorical_indicator_out, attribute_names_out = data_pre_processing.get('train')

In [ ]:
X_out

In [ ]:
y_out

In [ ]:
categorical_indicator_out

In [ ]:
data_pre_processing.events